In [1]:
#RoBERTa is BERT variation but following changes
''' -Use dynamic mask, instead of static mask
     -Remove the NSP task and train using the MLM task
     -Train with large batch size
     -use byte-level BPE(BBPE) as a tokenizer
     '''

' -Use dynamic mask, instead of static mask\n     -Remove the NSP task and train using the MLM task\n     -Train with large batch size\n     -use byte-level BPE(BBPE) as a tokenizer\n     '

In [2]:
#SOP basically a classification task, classify pair of words belongs to positive or negative class(Order of word not swapped)
#or negative class(order of word swapped)

In [3]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util

In [4]:
roberta=RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)
roberta.config

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [5]:
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\NLP-Dataset.csv"
df=pd.read_csv(path,sep=None,engine='python')

In [7]:
train_data=df['sentence'].loc[0:2]
test_data=df['sentence'].loc[3:]
train_data

0    My name is uttam asodariya
1        I am ambitious student
2             I like travelling
Name: sentence, dtype: object

In [8]:
def Word_embedding(dataframe):
    encoded_tensor=[]
    for i in dataframe:
        input = torch.tensor(tokenizer.encode(i, add_special_tokens=True)).unsqueeze(0)
        
        
        with torch.no_grad():
            output = roberta(input)
            last_hidden_states=output[0]
         
        last_hidden_states=np.array(last_hidden_states)
        sum_of_encoders=np.sum(last_hidden_states,axis=0)
        tensor=sum_of_encoders/12 
# we have 12 encoder output(length of 768) per token. Therefor average it
        encoded_tensor.append(tensor)
    return encoded_tensor

         
encoded_tensor=Word_embedding(train_data)      
encoded_tensor

[array([[-0.00380495,  0.0066697 , -0.0023425 , ..., -0.00875238,
         -0.00319631, -0.00455428],
        [-0.00872644,  0.00553709,  0.00703505, ..., -0.00305844,
         -0.00150916, -0.01476572],
        [-0.00223564,  0.006395  ,  0.00709376, ..., -0.02263761,
         -0.001949  , -0.01004509],
        ...,
        [-0.00790431,  0.00265047,  0.01174918, ..., -0.03812897,
         -0.01115266, -0.0071437 ],
        [ 0.00882291,  0.00030284,  0.00789095, ..., -0.02823584,
         -0.0073701 , -0.01307125],
        [-0.00331656,  0.00618941, -0.00465911, ..., -0.01265754,
         -0.00314743, -0.00760065]], dtype=float32),
 array([[-0.00288601,  0.00594991, -0.00174751, ..., -0.00520786,
         -0.00245082, -0.00411506],
        [-0.00365515,  0.00737569, -0.0091724 , ...,  0.02620407,
          0.00355022, -0.01840989],
        [ 0.00868383,  0.01075837,  0.01084779, ...,  0.00615936,
          0.00550099, -0.01763579],
        [ 0.00626601, -0.01360092, -0.00530282, ...,

In [9]:
def cosine(encoded_tensor): 
    encoded_tensor1=encoded_tensor[0]
    encoded_tensor2=encoded_tensor[1]
    
    print('cosine similarity between sentence 1&2', cosine_similarity(encoded_tensor1,
                                                                        encoded_tensor2))
